In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import branca
import folium
from shared_utils import gtfs_utils_v2
from segment_speed_utils import (helpers, 
                                 gtfs_schedule_wrangling, 
                                )

import pandas as pd
import geopandas as gpd 

import datetime as dt
import time
import fsspec
import gcsfs
from calitp_data_analysis import get_fs
fs = get_fs()
import tempfile

In [2]:
#Setting analysis date
analysis_dt = dt.date(2025,6,2)

In [3]:
# #Function to fetch feeds, trips, stoptimes and stops_geo data from warehouse v2
# def get_feeds_trips_stops_data(selected_date):
    
#     trip_cols = ["name", "gtfs_dataset_key", "feed_key", "trip_id", "route_id", "route_type", "route_key", 
#                  "num_stop_times", "direction_id", "trip_first_departure_sec", "trip_first_departure_ts", 
#                  "trip_last_arrival_sec", "trip_start_timezone", "trip_instance_key", "service_hours", "trip_first_departure_datetime_pacific"]
#     stoptimes_cols = ["key", "_gtfs_key", "feed_key", "trip_id", "stop_id", "stop_sequence", "arrival_time", "departure_time",
#        "arrival_time_interval", "departure_time_interval", "arrival_hour", "departure_hour"]
#     stop_cols = ["feed_key", "stop_id", "geometry", "stop_name", "stop_code", "location_type", "stop_desc"]
    
#     feed_data = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=selected_date)
    
    
#     feed_key_list = feed_data['feed_key'].tolist()
    
#     trips_data_list = []
#     stoptimes_data_list = []
#     stop_locations_gdf = gpd.GeoDataFrame()
#     for feed_key in feed_key_list:
#         try:
#             trips = gtfs_utils_v2.get_trips(selected_date=selected_date, operator_feeds=[feed_key])[trip_cols]
#             trips_data_list.append(trips)

#             stoptimes = gtfs_utils_v2.get_stop_times(selected_date=selected_date, operator_feeds=[feed_key], 
#                                                     trip_df=trips, get_df=True)[stoptimes_cols]
#             stoptimes_data_list.append(stoptimes)

#             stops_gdf = gtfs_utils_v2.get_stops(selected_date=selected_date, operator_feeds=[feed_key])[stop_cols]
#             stop_locations_gdf = pd.concat([stop_locations_gdf, stops_gdf], ignore_index=True)
#         except:
#             print('Skipping a row')
    
#     trips_data = pd.concat(trips_data_list, ignore_index=True)
#     stoptimes_data = pd.concat(stoptimes_data_list, ignore_index=True)
    
#     return feed_data, trips_data, stoptimes_data, stop_locations_gdf

In [4]:
# feed_data, trips_data, stoptimes_data, stop_locations_gdf = get_feeds_trips_stops_data(analysis_dt)

In [5]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/ahsc_grant'

Exporting data to GCS for reuse, minimizing repeated warehouse queries.

In [6]:
# trips_data.to_parquet(f"{GCS_FILE_PATH}/trips_06_02_2025.parquet")
# stoptimes_data.to_csv(f"{GCS_FILE_PATH}/stoptimes_06_02_2025.csv")
# feed_data.to_parquet(f"{GCS_FILE_PATH}/feed_06_02_2025.parquet")
# with fsspec.open(f"{GCS_FILE_PATH}/stop_locations_06_02_2025.parquet", 'wb') as f:
#     stop_locations_gdf.to_parquet(f)

Reading stoptimes, trips, feed and stop locations data 

In [7]:
stoptimes_data = pd.read_csv(f"{GCS_FILE_PATH}/stoptimes_06_02_2025.csv")

/tmp/ipykernel_1677/658927763.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  stoptimes_data = pd.read_csv(f"{GCS_FILE_PATH}/stoptimes_06_02_2025.csv")


In [8]:
trips_data = pd.read_parquet(f"{GCS_FILE_PATH}/trips_06_02_2025.parquet")

In [9]:
feed_data = pd.read_parquet(f"{GCS_FILE_PATH}/feed_06_02_2025.parquet")

In [10]:
with fs.open(f"{GCS_FILE_PATH}/stop_locations_06_02_2025.parquet", 'rb') as f:
    stop_locations_gdf = gpd.read_parquet(f)

In [11]:
# Retaining California based stops only 
# Loading California county boundaries GeoJSON data from ArcGIS REST service
CA_URL = "https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_County_Boundaries/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
CA_county = gpd.read_file(CA_URL)

In [12]:
CA_county = CA_county.to_crs(stop_locations_gdf.crs)

In [13]:
# Spatially joining stop locations with California county boundaries to keep stops within CA
stop_locations_gdf_ca = gpd.sjoin(
    stop_locations_gdf,  
    CA_county,                    
    how='inner',
    predicate='intersects'
)

In [14]:
stop_locations_gdf_ca.shape

(84082, 18)

In [15]:
stoptimesdata_california = pd.merge(
    stoptimes_data,
    stop_locations_gdf_ca[['feed_key', 'stop_id', 'stop_name', 'location_type', 'stop_desc', 'geometry']],
    on=['feed_key', 'stop_id'],
    how='inner'
)

In [16]:
stoptimesdata_california.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126889 entries, 0 to 1126888
Data columns (total 17 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   Unnamed: 0               1126889 non-null  int64   
 1   key                      1126889 non-null  object  
 2   _gtfs_key                1126889 non-null  object  
 3   feed_key                 1126889 non-null  object  
 4   trip_id                  1126889 non-null  object  
 5   stop_id                  1126889 non-null  object  
 6   stop_sequence            1126889 non-null  int64   
 7   arrival_time             1061034 non-null  object  
 8   departure_time           1061034 non-null  object  
 9   arrival_time_interval    1061034 non-null  object  
 10  departure_time_interval  1061034 non-null  object  
 11  arrival_hour             1061034 non-null  float64 
 12  departure_hour           1061034 non-null  float64 
 13  stop_name                11

In [17]:
trips_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162083 entries, 0 to 162082
Data columns (total 16 columns):
 #   Column                                 Non-Null Count   Dtype              
---  ------                                 --------------   -----              
 0   name                                   162083 non-null  object             
 1   gtfs_dataset_key                       162083 non-null  object             
 2   feed_key                               162083 non-null  object             
 3   trip_id                                162083 non-null  object             
 4   route_id                               162083 non-null  object             
 5   route_type                             162083 non-null  object             
 6   route_key                              162083 non-null  object             
 7   num_stop_times                         162083 non-null  int64              
 8   direction_id                           94257 non-null   float64           

In [18]:
trips_data_cleaned = trips_data.drop_duplicates() # Dropping duplicates

In [19]:
trips_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162072 entries, 0 to 162082
Data columns (total 16 columns):
 #   Column                                 Non-Null Count   Dtype              
---  ------                                 --------------   -----              
 0   name                                   162072 non-null  object             
 1   gtfs_dataset_key                       162072 non-null  object             
 2   feed_key                               162072 non-null  object             
 3   trip_id                                162072 non-null  object             
 4   route_id                               162072 non-null  object             
 5   route_type                             162072 non-null  object             
 6   route_key                              162072 non-null  object             
 7   num_stop_times                         162072 non-null  int64              
 8   direction_id                           94246 non-null   float64           

In [20]:
#Merging stop data with trip data on trip id and feed key.
stop_trip_merged = pd.merge(stoptimesdata_california,
                            trips_data_cleaned,
                            on = ['trip_id', 'feed_key'],
                            how = 'left')

In [28]:
# Adding peak/off-peak labels to trip instances based on scheduled time buckets for the given analysis date
sched_time_of_day = gtfs_schedule_wrangling.get_trip_time_buckets(
    analysis_dt
).pipe(
    gtfs_schedule_wrangling.add_peak_offpeak_column
)[["trip_instance_key", "peak_offpeak"]]

In [29]:
#Merging scheduled time of the data 
stop_trip_merged = pd.merge(
    stop_trip_merged,
    sched_time_of_day,
    on='trip_instance_key',
    how='left'
)

In [21]:
# Counting unique trips per stop and renaming column to 'num_trips'
num_trips_per_stop = stop_trip_merged.groupby('stop_id')['trip_instance_key'].nunique().reset_index().rename(columns={'trip_instance_key':'num_trips'})

In [22]:
num_trips_per_stop.shape

(28350, 2)

In [23]:
# Counting unique routes per stop and renaming column to 'num_routes'
num_routes_per_stop = stop_trip_merged.groupby('stop_id')['route_id'].nunique().reset_index().rename(columns={'route_id':'num_routes'})

In [24]:
num_routes_per_stop.shape

(28350, 2)

In [34]:
# Filtering stop-trip data to include only peak period trips
peak_stop_times = stop_trip_merged[stop_trip_merged['peak_offpeak'] == 'peak'].copy()

In [35]:
#Filter for peak + arrival before 9 AM
am_peak_stop_trip = peak_stop_times[peak_stop_times['arrival_hour'] < 9].copy()

#Assign time_of_day manually
am_peak_stop_trip['time_of_day'] = 'AM Peak'


In [51]:
group_cols = ['feed_key','stop_id', 'route_id', 'direction_id']

In [52]:
stop_route_summary = gtfs_schedule_wrangling.aggregate_time_of_day_to_peak_offpeak(
    df=am_peak_stop_trip,
    group_cols=group_cols,
    long_or_wide='wide'
)

In [53]:
stop_route_summary['peak_headway'] = 60 / stop_route_summary['peak_frequency']

In [54]:
stop_route_summary.head(5)

,feed_key,stop_id,route_id,direction_id,all_day_n_trips,peak_n_trips,all_day_frequency,peak_frequency,peak_headway
0,058a3b43698f803c8686f7e72a9efd4c,121914,11796,1.0,1.0,1.0,0.04,0.12,500.0
1,058a3b43698f803c8686f7e72a9efd4c,121915,11792,1.0,1.0,1.0,0.04,0.12,500.0
2,058a3b43698f803c8686f7e72a9efd4c,121916,11792,1.0,1.0,1.0,0.04,0.12,500.0
3,058a3b43698f803c8686f7e72a9efd4c,121917,11792,1.0,1.0,1.0,0.04,0.12,500.0
4,058a3b43698f803c8686f7e72a9efd4c,121919,11795,1.0,2.0,2.0,0.08,0.25,240.0


In [ ]:
# # Calculating average AM peak headway (minutes) for each stop
# headways_per_stop = (
#     am_peak_times_sorted
#     .groupby('stop_id')['peak_headway']
#     .mean()  
#     .reset_index()
# )

In [37]:
# # Saving geojson file
# with tempfile.NamedTemporaryFile(suffix=".geojson") as tmp:
#     stop_summary_final.to_file(tmp.name, driver="GeoJSON")


#     with fsspec.open(f"{GCS_FILE_PATH}/stop_summary.geojson", 'w') as f_out:
#         with open(tmp.name, 'r') as f_in:
#             f_out.write(f_in.read())